In [1]:
import boto3
import time

In [3]:
import pandas as pd

In [2]:
import pandas as pd
from io import StringIO

In [6]:
import configparser

In [7]:
config = configparser.ConfigParser()

In [16]:
config.read_file(open('cluster3.config'))

In [17]:
key = config['AWS']['KEY']
secret = config['AWS']['SECRET']
region = config['DWH']['REGION']

In [19]:
athenaClient = boto3.client(
    "athena",
    aws_access_key_id=key,
    aws_secret_access_key=secret,
    region_name=region)

In [20]:
Dict = {}
def download_and_load_query_results(client, query_response):
    while True:
        try:
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(2)
            else:
                raise err
                
    temp_file_location = "athena_query_results.csv"
    s3_client = boto3.client("s3",
                            aws_access_key_id=AWS_ACCESS_KEY,
                            aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                            region_name=AWS_REGION)
    s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location)
    return pd.read_csv(temp_file_location)

### After Glue has catalogued

The next steps are

1) Query the Tables using Athena Client

2) Establish connection to local database using psycopg2

3) Move the data from athena to local data using the Jupyter NB interface

4) Do the DB refactoring on JP notebook

5) After completion upload the final file to S3

**Challenge is the crawler is not fetching the data from the files. It is only fetching the column names. Need to check the process 

In [37]:
response = athenaClient.start_query_execution(
    QueryString = "SELECT * FROM enigma_jhu;",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"}    
    })

In [4]:
csv_buff = StringIO()

In [87]:
s3Reso.Object(bucket,'output/dimHospital.csv').put(Body=csv_buff.getvalue())

{'ResponseMetadata': {'RequestId': 'DR9WZPDT7AJSRNKK',
  'HostId': 'SmuwdPDosEwW0yMerj62GpmeBAf95ETk0TkiNqcs7dSOLc/m6BGMQI3GscyNPqmR0/Q6Rkvdma8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'SmuwdPDosEwW0yMerj62GpmeBAf95ETk0TkiNqcs7dSOLc/m6BGMQI3GscyNPqmR0/Q6Rkvdma8=',
   'x-amz-request-id': 'DR9WZPDT7AJSRNKK',
   'date': 'Sun, 30 Oct 2022 09:53:49 GMT',
   'etag': '"052b91a93c128f0920be05c2066e5f13"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"052b91a93c128f0920be05c2066e5f13"'}

In [88]:
dimRegion2.to_csv(csv_buff)

In [29]:
s3 = boto3.resource('s3',
                    region_name="us-east-1",
                    aws_access_key_id=key,
                    aws_secret_access_key=secret)

In [30]:
bucket = s3.Bucket("airbnb-fresh")

In [31]:
locFile = [filename.key for filename in bucket.objects.filter(Prefix='')]

In [32]:
locFile

['calendar.csv',
 'listings.csv',
 'listings_details.csv',
 'neighbourhoods.csv',
 'reviews.csv',
 'reviews_details.csv']

In [33]:
import psycopg2

In [113]:
try:
    conn = psycopg2.connect(host=DWH_ENDPOINT,dbname=DB_NAME,user=DB_USER,password=DWH_DB_PASSWORD,port=DWH_PORT)
except Exception as e:
    print(e)

In [114]:
conn.set_session(autocommit=True)

In [115]:
try:
    cur = conn.cursor()
    
except:
    print(e)

In [133]:
try:
    cur.execute("""
        CREATE TABLE "factcorrected" (
            "index" INTEGER,
          "fips" REAL,
          "province_state" TEXT,
          "country_region" TEXT,
          "confirmed" REAL,
          "deaths" REAL,
          "recovered" REAL,
          "active" REAL,
          "date" INTEGER,
          "state" TEXT,
          "positive" REAL,
          "negative" REAL,
          "hospitalizedcurrently" REAL,
          "hospitalized" REAL,
          "hospitalizeddischarged" REAL
    )
    """)
except Exception as e:
    print(e)

In [125]:
DWH_ROLE_ARN

'arn:aws:iam::642924624251:role/redshiftRole'

In [134]:
try:
    cur.execute("""
        COPY factcorrected from 's3://de-glued/output/factcovid.csv'
        CREDENTIALS 'aws_iam_role=arn:aws:iam::642924624251:role/redshiftRole'
        DELIMITER ','
        region 'us-east-1'
    """)
except Exception as e:
    print(e)

Load into table 'factcorrected' failed.  Check 'stl_load_errors' system table for details.

